## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,TimestampType,FloatType,DoubleType, ArrayType, StringType
from pyspark.sql.functions import input_file_name, current_timestamp, col, count, isnan, when, desc, sum as sqlsum, regexp_extract, regexp_replace, to_timestamp, udf
import mlflow.pyfunc
import re


## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
tfiles = dbutils.fs.ls(TWEET_SOURCE_PATH)
count = len(tfiles)
print("No. of Source files:",count)

In [0]:
display(tfiles)

In [0]:
print(dbutils.fs.head("s3a://voc-75-databricks-data/voc_volume/10000.json"))


## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# Schema for the raw data
raw_data_schema = StructType([
        StructField("date", StringType(), True),
        StructField("user", StringType(), True),
        StructField("text", StringType(), True),
        StructField("sentiment", StringType(), True),
        StructField("source_file", StringType(), True),
        StructField("processing_time",TimestampType(), True)
])

In [0]:
# read streaming raw data files

raw_data_df = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("mergeSchema","true")
    .schema(raw_data_schema) 
    .load(TWEET_SOURCE_PATH)
    .withColumn("source_file",input_file_name()) 
    .withColumn("processing_time",current_timestamp())
)

In [0]:
# Write Stream

bronze_stream = (raw_data_df
                 .writeStream
                 .format("delta")
                 .option("checkpointLocation", BRONZE_CHECKPOINT)
                 .outputMode("append")
                 .start(BRONZE_DELTA))

In [0]:
bronze_stream.lastProgress

In [0]:
display(raw_data_df, streamName = "raw_data")

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
#no. of tweets captured in bronze table

# load the delta bronze table
bronze_df = spark.read.format("delta").load(BRONZE_DELTA)


tweet_count = bronze_df.count()
print(f"Total number of tweets: {tweet_count}")

In [0]:
from pyspark.sql.functions import count, when, col

# Counting the no. of NaN and Null
nan_null_counts = bronze_df.select([
    count(when(col(c).isNull(), c)).alias(c) for c in bronze_df.columns
])
display(nan_null_counts)

In [0]:
%sql
OPTIMIZE BRONZE_DELTA

In [0]:
# No. of Tweets by Unique User
user_tweet_counts = bronze_df.groupBy("user").count().orderBy(desc("count"))
display(user_tweet_counts)

In [0]:
# no. of tweets that mention atleast once or no mentions
mentions_df = bronze_df.withColumn("has_mention", when(col("text").contains("@"), 1).otherwise(0))
mention_counts = mentions_df.groupBy().agg(
    sqlsum("has_mention").alias("tweets_with_atleast_one_mention"),
    (count("text") - sqlsum("has_mention")).alias("tweets_without_mentions")
)
display(mention_counts)

In [0]:

# displaying top 20 twitter users
top20_tweeters = user_tweet_counts.limit(20)

display(top20_tweeters)




Databricks visualization. Run in Databricks to view.

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# Schema for the silver table
silver_schema = StructType([
    StructField("timestamp", TimestampType(), True),
    StructField("mention", StringType(), True),
    StructField("cleaned_text", StringType(), True),
    StructField("sentiment", StringType(), True)
])

In [0]:
bronze_df = spark.readStream.format("delta").load(BRONZE_DELTA)

In [0]:
# extract mentions
from pyspark.sql.functions import udf, regexp_extract
from pyspark.sql.types import StringType

def extract_mentions(text):
    mention = ' '.join(set(part[1:] for part in text.split() if part.startswith('@')))
    return mention

extract_mentions_udf = udf(extract_mentions, StringType())


In [0]:
# doing clean texts
import nltk
from nltk.stem import WordNetLemmatizer
import string
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [0]:
from pyspark.ml.feature import StopWordsRemover
def clean_text(text):
    from nltk.corpus import stopwords
    # Initialize the lemmatizer
    nltk.download('punkt')
    nltk.download('wordnet')
    nltk.download('stopwords')

    lemmatizer = WordNetLemmatizer()
   
    # Get English stopwords
    stop_words = set(stopwords.words('english'))

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
   
    # Lemmatize words, remove stopwords and extra spaces
    # Tokenize the text, lemmatize each word, and remove stopwords
    words = nltk.word_tokenize(text)
    lemmatized_text = ' '.join([lemmatizer.lemmatize(word).lower() for word in words if word.lower() not in stop_words])
   
    return lemmatized_text
 
clean_text_udf = udf(clean_text, StringType())

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re

def remove_mentions(text):
    if text is not None:
        return re.sub(r"@\w+", "", text)
    else:
        return None


remove_mentions_udf = udf(remove_mentions, StringType())


In [0]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import StringType
import pandas as pd
import re

@pandas_udf(StringType())
def remove_mentions_pandas_udf(text_series: pd.Series) -> pd.Series:
    # Apply regex to each item in the Series
    return text_series.apply(lambda text: re.sub(r"@\w+", "", text) if text is not None else text)


In [0]:
from pyspark.sql.functions import col, regexp_replace, to_timestamp, udf, expr
from pyspark.sql.types import StringType

# Convert timestamp from string to timestamp
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
bronze_df = bronze_df.withColumn("timestamp", to_timestamp(col("date"), "EEE MMM dd HH:mm:ss zzz yyyy"))


# Extract mentions
bronze_df = bronze_df.withColumn("mention", extract_mentions_udf(col("text")))

# Filter records with at least one mention
bronze_df = bronze_df.filter(col("mention").isNotNull() & (col("mention") != "") & (col("mention") != " "))

# Clean text by removing mentions
# bronze_df = bronze_df.withColumn("cleaned_text", regexp_replace(col("text"), r"@\w+", ""))
#bronze_df = bronze_df.withColumn("cleaned_text", remove_mentions_udf(col("text")))
#bronze_df = bronze_df.withColumn("cleaned_text", remove_mentions_pandas_udf(col("text")))
bronze_df = bronze_df.withColumn("cleaned_text",clean_text_udf(col("text")))



In [0]:
display(bronze_df)

In [0]:
 #dbutils.fs.rm(SILVER_DELTA, recurse=True)
 #dbutils.fs.rm(SILVER_CHECKPOINT, recurse=True)

In [0]:
# Write stream in silver delta table
silver_stream = bronze_df.select("timestamp", "mention", "cleaned_text", "sentiment") \
    .writeStream \
    .format("delta") \
    .option("checkpointLocation", SILVER_CHECKPOINT) \
    .outputMode("append") \
    .start(SILVER_DELTA)

In [0]:
silver_df = spark.read.format("delta").load(SILVER_DELTA)
display(silver_df)

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
# Read streaming the silver table
silver_df = spark.readStream.format("delta").load(SILVER_DELTA)


In [0]:
# Loading the Hugging face model from ML flow
model_path = f"models:/{MODEL_NAME}/Production"

sentiment_model = mlflow.pyfunc.load_model(model_path)

In [0]:
# %sql
# OPTIMIZE SILVER_DELTA

In [0]:
#dbutils.fs.rm(GOLD_DELTA, recurse=True)
#dbutils.fs.rm(GOLD_CHECKPOINT, recurse=True)

In [0]:
from pyspark.sql.functions import pandas_udf, PandasUDFType, col, when
import pandas as pd
import mlflow.pyfunc
 
# Assuming the model is loaded correctly and available as sentiment_model
# Load the model outside the UDF, globally if possible to avoid reloading per task
#sentiment_model = mlflow.pyfunc.load_model("models:/your_model_name/Production")
 
@pandas_udf("struct<sentiment_label:string, sentiment_score:double, sentiment_id:int>", PandasUDFType.SCALAR)
def predict_sentiment(texts: pd.Series) -> pd.DataFrame:
    # Apply prediction to each text entry
    results = texts.apply(lambda text: sentiment_model.predict([text]))
 
    # Extract data into separate series
    labels = results.apply(lambda result: result['label'][0])
    scores = results.apply(lambda result: result['score'][0])
    sentiment_ids = labels.apply(lambda label: 1 if label == 'POS' else 0)
 
    return pd.DataFrame({
        'sentiment_label': labels,
        'sentiment_score': scores,
        'sentiment_id': sentiment_ids
    })

In [0]:
gold_df = silver_df.withColumn("sentiment_result", predict_sentiment(col("cleaned_text")))
gold_df = gold_df.select(
    col("timestamp"),
    col("mention"),
    col("cleaned_text"),
    col("sentiment"),
    col("sentiment_result.sentiment_score").alias("predicted_score"),
    col("sentiment_result.sentiment_label").alias("predicted_sentiment"),
    col("sentiment_result.sentiment_id").alias("predicted_sentiment_id")
)\
.withColumn("sentiment_id", when(col("sentiment") == "positive", 1).otherwise(0))

In [0]:
# Writing in from silver table with updated values into gold delta table
gold_stream = gold_df.writeStream \
    .format("delta") \
    .option("checkpointLocation", GOLD_CHECKPOINT) \
    .outputMode("append") \
    .trigger(processingTime='1 millisecond') \
    .start(GOLD_DELTA)

In [0]:
gold_stream.status

In [0]:
display(gold_df)

In [0]:
gold_delta_table = spark.read.format("delta").load(GOLD_DELTA)

In [0]:
display(gold_delta_table)

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

In [0]:
# Upload the gold stream data
gold_df = spark.read.format("delta").load(GOLD_DELTA)

In [0]:
import pandas as pd

# Assuming 'sentiment_id' is the actual labels and 'predicted_sentiment_id' is the predictions
pdf = gold_df.select("sentiment_id", "predicted_sentiment_id").toPandas()

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate metrics
precision = precision_score(pdf['sentiment_id'], pdf['predicted_sentiment_id'])
recall = recall_score(pdf['sentiment_id'], pdf['predicted_sentiment_id'])
f1 = f1_score(pdf['sentiment_id'], pdf['predicted_sentiment_id'])

# Calculate confusion matrix
conf_matrix = confusion_matrix(pdf['sentiment_id'], pdf['predicted_sentiment_id'])
ipath = "/Workspace/Repos/labuser104917-3025803@vocareum.com/dscc202-402-spring2024/final_project/confusion_matrix.png"
# Plotting the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.savefig(ipath)
plt.show()


In [0]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
import mlflow

path = "/Workspace/Repos/labuser104917-3025803@vocareum.com/dscc202-402-spring2024/final_project/confusion_matrix.png"

with mlflow.start_run():
    mlflow.log_param("Model Name",MODEL_NAME)
    mlflow.log_param("MLflow Version", mlflow.__version__)
    silver_version = spark.sql("DESCRIBE HISTORY silver_delta").select("version").head()[0]
    mlflow.log_param("silver_delta_version", silver_version)
    mlflow.log_metric("Precision", precision)
    mlflow.log_metric("Recall", recall)
    mlflow.log_metric("F1 Score", f1)
    mlflow.log_artifact(f"{path}")


## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
from pyspark.sql.functions import countDistinct

mention_count = gold_df.agg(countDistinct("mention").alias("total_mentions"))
mention_count.show()


In [0]:
from pyspark.sql.functions import col, count, when

sentiment_counts = gold_df.groupBy("mention").agg(
    count(when(col("predicted_sentiment") == "NEU", True)).alias("neutral_count"),
    count(when(col("predicted_sentiment") == "POS", True)).alias("positive_count"),
    count(when(col("predicted_sentiment") == "NEG", True)).alias("negative_count")
)

sorted_by_total = sentiment_counts.withColumn("total", col("neutral_count") + col("positive_count") + col("negative_count"))
sorted_by_total = sorted_by_total.orderBy(col("total").desc())
sorted_by_total.show()

In [0]:
top_pos = sorted_by_total[['mention','positive_count']].sort(col('positive_count').desc()).limit(20)
display(top_pos)

Databricks visualization. Run in Databricks to view.

In [0]:
top_neg = sorted_by_total[['mention','negative_count']].sort(col('negative_count').desc()).limit(20)
display(top_neg)

Databricks visualization. Run in Databricks to view.

## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# ENTER YOUR CODE HERE

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE